In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)
pd.options.mode.chained_assignment = None

# student spreadsheet!

jobs = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS8YLSz7h6ZcGKd9k62ET5IjCMPq-Jq3J6iYbJRwwD7GEoSG6i4GXp1nDOePX7m1ri7-65Qn-CjUqND/pub?output=csv')
jobs

,job_title,job_url,job_description
0,Candidate Services Coordinator,https://www.governmentjobs.com/jobs/4814083-0/candidate-services-coordinator,"The Candidate Services Coordinator is a key member of the Supervisor of Elections Office, responsible for ensuring smooth administrative operations with a focus on candidate services and assists with accounting functions and payroll processing. This position requires attention to detail, organiz..."
1,County Clerk,https://www.governmentjobs.com/careers/lanecountyor/jobs/4777148/county-clerk-elections-recording-principal-manager,"Lane County is seeking an experienced and dynamic leader to serve as County Clerk and Election & Recording Principal Manager, overseeing critical functions that uphold the integrity of our democratic processes and public records. This pivotal role leads a dedicated team committed to excellence, ..."
2,Customer Support Consultant,https://www.hartintercivic.com/jobs/customer-support-center-consultant-level-1/,"A Customer Support Center Consultant Level 1 (CSC 1) responds to requests for assistance from Hart InterCivic customers for all Hart InterCivic products. The CSC I’s primary responsibilities are to: Answer, resolve and route customer queries; Maintain professionalism and “customer first” approac..."
3,Deputy Director,https://mn.gov/mmb/careers/search-for-jobs/,"The Office of the Minnesota Secretary of State (OSS) is seeking a Deputy Elections Director that supports the Elections Director in providing division-wide strategic and administrative management, along with program and staff management, of all activities within the Elections Division to ensure ..."
4,Election Specialist,https://www.votepalmbeach.gov/About-Us/Election-Specialist-Candidate-Services,"This position is responsible for the management and execution of services provided to candidates, political committees, electioneering communication organizations, political parties, community development districts, and special taxing districts. This includes establishing and maintaining an orga..."
5,Elections Specialist,https://www.governmentjobs.com/careers/pinalcounty/jobs/4820593/elections-specialist?keywords=election&pagetype=jobOpportunitiesJobs,Perform a variety of paraprofessional and technical functions in the administration and support of the elections functions for Pinal County under general supervision. Work in this classification requires an individual to be able to perform the essential job functions satisfactorily. Reasonable ...
6,Elections Systems Administrator,https://www.governmentjobs.com/careers/arapahoe/jobs/4878211/elections-system-administrator?sort=PostingDate%7CDescending&pagetype=jobOpportunitiesJobs,"The Election Systems Administrator performs leadership level administrative, and professional work in carrying out a comprehensive public facing service operation. This position specifically leads and supports all areas of ballot processing, ballot auditing and election technology maintenance.Sa..."
7,IT Assistant Manager,https://www.votepalmbeach.gov/About-Us/IT-Assistant-Manager,"The Assistant IT Manager plays a supportive role in the smooth operation of the IT department, ensuring that both the technical infrastructure and the team are aligned with the organization’s goals. This position involves collaborating closely with the Election Technology Director to oversee the..."
8,Language Access & Outreach Coordinator,https://www.governmentjobs.com/careers/kingcounty/jobs/4788084/language-access-and-outreach-coordinator-chinese?department[0]=KCE%20-%20King%20County%20Elections&sort=PositionTitle%7CAscending&pagetype=jobOpportunitiesJobs,"This is an amazing opportunity to be engaged in the election process! The Department of Elections is searching for energetic and resourceful professionals who like to “get stuff done”. The Language Access and Outreach Coordinator position in the Elections Department combines an exciting, fast p..."
9,Project Manager,https://www.hartintercivic.com/

In [34]:
desc = jobs.loc[0, 'job_description']
desc

'The Candidate Services Coordinator is a key member of the Supervisor of Elections Office, responsible for ensuring smooth administrative operations with a focus on candidate services and assists with accounting functions and payroll processing. This position requires attention to detail, organizational skills, and the ability to effectively handle multiple responsibilities while maintaining confidentiality and compliance with applicable laws and regulations. This position is responsible for managing the candidate qualifying process, serving as primary point of contact for candidates, maintaining accurate records related to candidate filings, financial disclosures, and other required documentation, and coordinating candidate workshops and providing training on election procedures and campaign finance rules.Salary:$36,000 – $46,000 Annually.Application:For the complete job listing and to apply,click here.'

In [ ]:
def get_salary(description):
    return int(description.split('$')[1].split(' ')[0].replace(',', ''))
# this works but is not looking great! it's brittle.

get_salary(desc)

36000

In [ ]:
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps

# ---------- FREE API KEY examples ---------------------

owm = OWM('YOUR API KEY HERE')
mgr = owm.weather_manager()


# Search for current weather in London (Great Britain) and get details
observation = mgr.weather_at_place('Washington,DC,USA')
w = observation.weather

w.detailed_status         # 'clouds'
w.wind()                  # {'speed': 4.6, 'deg': 330}
w.humidity                # 87
w.temperature('celsius')  # {'temp_max': 10.5, 'temp': 9.7, 'temp_min': 9.0}
w.rain                    # {}
w.heat_index              # None
w.clouds                  # 75

w.temperature('fahrenheit')
w.clouds
# # Will it be clear tomorrow at this time in Milan (Italy) ?
# forecast = mgr.forecast_at_place('Milan,IT', 'daily')
# answer = forecast.will_be_clear_at(timestamps.tomorrow())

# # ---------- PAID API KEY example ---------------------

# config_dict = config.get_default_config_for_subscription_type('professional')
# owm = OWM('your paid OWM API key', config_dict)

# # What's the current humidity in Berlin (Germany) ?
# one_call_object = mgr.one_call(lat=52.5244, lon=13.4105)
# one_call_object.current.humidity

100

In [ ]:
import json
from openai import OpenAI

client = OpenAI()

def parse_job_description(job_description):
    response = client.responses.create(
        model="gpt-4o-2024-08-06",
        input=[
            {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a job description posting and should convert it into the given structure. Omit $ signs. DO this do that"},
            {"role": "user", "content": job_description},
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "research_paper_extraction",
                "schema": {
                    "type": "object",
                    "properties": {
                        "salary_low_end": { "type": "string" },
                        "salary_high_end": { "type": "string" },
                        "employer": { "type": "string" },
                        "location": { "type": "string" },
                        "deadline": { "type": "string" },
                        "nonprofit_or_government": { "type": "string" },
                        "responsible_for_administering_elections": { "type": "boolean" }
                    },
                    "required": ["salary_low_end", "salary_high_end", "employer", "location", "deadline", "nonprofit_or_government", "responsible_for_administering_elections"],
                    "additionalProperties": False
                },
                "strict": True
            },
        },
    )

    output = json.loads(response.output_text)
    return output

In [60]:
parse_job_description(desc)

{'salary_low_end': '36000',
 'salary_high_end': '46000',
 'employer': 'Supervisor of Elections Office',
 'location': '',
 'deadline': '',
 'nonprofit_or_government': 'Government',
 'responsible_for_administering_elections': True}

In [55]:
desc

'The Candidate Services Coordinator is a key member of the Supervisor of Elections Office, responsible for ensuring smooth administrative operations with a focus on candidate services and assists with accounting functions and payroll processing. This position requires attention to detail, organizational skills, and the ability to effectively handle multiple responsibilities while maintaining confidentiality and compliance with applicable laws and regulations. This position is responsible for managing the candidate qualifying process, serving as primary point of contact for candidates, maintaining accurate records related to candidate filings, financial disclosures, and other required documentation, and coordinating candidate workshops and providing training on election procedures and campaign finance rules.Salary:$36,000 – $46,000 Annually.Application:For the complete job listing and to apply,click here.'

In [62]:
fruits = ['apple', 'orange', 'strawberry', 'avocado']
for item in fruits:
    print(item[0])

a
o
s
a


In [76]:
short = jobs.head(5)

In [81]:
short

,job_title,job_url,job_description
0,Candidate Services Coordinator,https://www.governmentjobs.com/jobs/4814083-0/candidate-services-coordinator,"The Candidate Services Coordinator is a key member of the Supervisor of Elections Office, responsible for ensuring smooth administrative operations with a focus on candidate services and assists with accounting functions and payroll processing. This position requires attention to detail, organiz..."
1,County Clerk,https://www.governmentjobs.com/careers/lanecountyor/jobs/4777148/county-clerk-elections-recording-principal-manager,"Lane County is seeking an experienced and dynamic leader to serve as County Clerk and Election & Recording Principal Manager, overseeing critical functions that uphold the integrity of our democratic processes and public records. This pivotal role leads a dedicated team committed to excellence, ..."
2,Customer Support Consultant,https://www.hartintercivic.com/jobs/customer-support-center-consultant-level-1/,"A Customer Support Center Consultant Level 1 (CSC 1) responds to requests for assistance from Hart InterCivic customers for all Hart InterCivic products. The CSC I’s primary responsibilities are to: Answer, resolve and route customer queries; Maintain professionalism and “customer first” approac..."
3,Deputy Director,https://mn.gov/mmb/careers/search-for-jobs/,"The Office of the Minnesota Secretary of State (OSS) is seeking a Deputy Elections Director that supports the Elections Director in providing division-wide strategic and administrative management, along with program and staff management, of all activities within the Elections Division to ensure ..."
4,Election Specialist,https://www.votepalmbeach.gov/About-Us/Election-Specialist-Candidate-Services,"This position is responsible for the management and execution of services provided to candidates, political committees, electioneering communication organizations, political parties, community development districts, and special taxing districts. This includes establishing and maintaining an orga..."


In [86]:
parsed['salary_high_end']

'$24.04'

In [89]:
parsed

{'salary_low_end': '21.63',
 'salary_high_end': '24.04',
 'employer': 'Not specified',
 'location': 'Not specified',
 'deadline': 'Not specified',
 'nonprofit_or_government': 'Not specified',
 'responsible_for_administering_elections': True}

In [90]:
for item in parsed:
    print(item)

salary_low_end
salary_high_end
employer
location
deadline
nonprofit_or_government
responsible_for_administering_elections


In [ ]:
for i, row in short.iterrows():
    # print(row['job_description'])
    desc = row['job_description']
    parsed = parse_job_description(desc)
    # print(parsed)
    # print('\n\n-----------------\n\n\n\n')
    for item in parsed:
        short.loc[i, item] = parsed[item]
    

The Candidate Services Coordinator is a key member of the Supervisor of Elections Office, responsible for ensuring smooth administrative operations with a focus on candidate services and assists with accounting functions and payroll processing. This position requires attention to detail, organizational skills, and the ability to effectively handle multiple responsibilities while maintaining confidentiality and compliance with applicable laws and regulations. This position is responsible for managing the candidate qualifying process, serving as primary point of contact for candidates, maintaining accurate records related to candidate filings, financial disclosures, and other required documentation, and coordinating candidate workshops and providing training on election procedures and campaign finance rules.Salary:$36,000 – $46,000 Annually.Application:For the complete job listing and to apply,click here.
{'salary_low_end': '$36,000', 'salary_high_end': '$46,000', 'employer': 'Supervisor

In [92]:
short

,job_title,job_url,job_description,salary_low_end,salary_high_end,employer,location,deadline,nonprofit_or_government,responsible_for_administering_elections
0,Candidate Services Coordinator,https://www.governmentjobs.com/jobs/4814083-0/candidate-services-coordinator,"The Candidate Services Coordinator is a key member of the Supervisor of Elections Office, responsible for ensuring smooth administrative operations with a focus on candidate services and assists with accounting functions and payroll processing. This position requires attention to detail, organiz...","$36,000","$46,000",Supervisor of Elections Office,,,government,True
1,County Clerk,https://www.governmentjobs.com/careers/lanecountyor/jobs/4777148/county-clerk-elections-recording-principal-manager,"Lane County is seeking an experienced and dynamic leader to serve as County Clerk and Election & Recording Principal Manager, overseeing critical functions that uphold the integrity of our democratic processes and public records. This pivotal role leads a dedicated team committed to excellence, ...","$90,625.60","$133,286.40",Lane County,"Lane County, Oregon",N/A,Government,True
2,Customer Support Consultant,https://www.hartintercivic.com/jobs/customer-support-center-consultant-level-1/,"A Customer Support Center Consultant Level 1 (CSC 1) responds to requests for assistance from Hart InterCivic customers for all Hart InterCivic products. The CSC I’s primary responsibilities are to: Answer, resolve and route customer queries; Maintain professionalism and “customer first” approac...",Not specified,Not specified,Hart InterCivic,Not specified,Not specified,No,False
3,Deputy Director,https://mn.gov/mmb/careers/search-for-jobs/,"The Office of the Minnesota Secretary of State (OSS) is seeking a Deputy Elections Director that supports the Elections Director in providing division-wide strategic and administrative management, along with program and staff management, of all activities within the Elections Division to ensure ...","78,508","113,086",Office of the Minnesota Secretary of State (OSS),Minnesota,April 7,Government,True
4,Election Specialist,https://www.votepalmbeach.gov/About-Us/Election-Specialist-Candidate-Services,"This position is responsible for the management and execution of services provided to candidates, political committees, electioneering communication organizations, political parties, community development districts, and special taxing districts. This includes establishing and maintaining an orga...",21.63,24.04,,,,,True


In [83]:
jobs.loc[0, 'job_quality'] = 'extremely good' 

In [84]:
jobs

,job_title,job_url,job_description,job_quality
0,Candidate Services Coordinator,https://www.governmentjobs.com/jobs/4814083-0/candidate-services-coordinator,"The Candidate Services Coordinator is a key member of the Supervisor of Elections Office, responsible for ensuring smooth administrative operations with a focus on candidate services and assists with accounting functions and payroll processing. This position requires attention to detail, organiz...",extremely good
1,County Clerk,https://www.governmentjobs.com/careers/lanecountyor/jobs/4777148/county-clerk-elections-recording-principal-manager,"Lane County is seeking an experienced and dynamic leader to serve as County Clerk and Election & Recording Principal Manager, overseeing critical functions that uphold the integrity of our democratic processes and public records. This pivotal role leads a dedicated team committed to excellence, ...",NaN
2,Customer Support Consultant,https://www.hartintercivic.com/jobs/customer-support-center-consultant-level-1/,"A Customer Support Center Consultant Level 1 (CSC 1) responds to requests for assistance from Hart InterCivic customers for all Hart InterCivic products. The CSC I’s primary responsibilities are to: Answer, resolve and route customer queries; Maintain professionalism and “customer first” approac...",NaN
3,Deputy Director,https://mn.gov/mmb/careers/search-for-jobs/,"The Office of the Minnesota Secretary of State (OSS) is seeking a Deputy Elections Director that supports the Elections Director in providing division-wide strategic and administrative management, along with program and staff management, of all activities within the Elections Division to ensure ...",NaN
4,Election Specialist,https://www.votepalmbeach.gov/About-Us/Election-Specialist-Candidate-Services,"This position is responsible for the management and execution of services provided to candidates, political committees, electioneering communication organizations, political parties, community development districts, and special taxing districts. This includes establishing and maintaining an orga...",NaN
5,Elections Specialist,https://www.governmentjobs.com/careers/pinalcounty/jobs/4820593/elections-specialist?keywords=election&pagetype=jobOpportunitiesJobs,Perform a variety of paraprofessional and technical functions in the administration and support of the elections functions for Pinal County under general supervision. Work in this classification requires an individual to be able to perform the essential job functions satisfactorily. Reasonable ...,NaN
6,Elections Systems Administrator,https://www.governmentjobs.com/careers/arapahoe/jobs/4878211/elections-system-administrator?sort=PostingDate%7CDescending&pagetype=jobOpportunitiesJobs,"The Election Systems Administrator performs leadership level administrative, and professional work in carrying out a comprehensive public facing service operation. This position specifically leads and supports all areas of ballot processing, ballot auditing and election technology maintenance.Sa...",NaN
7,IT Assistant Manager,https://www.votepalmbeach.gov/About-Us/IT-Assistant-Manager,"The Assistant IT Manager plays a supportive role in the smooth operation of the IT department, ensuring that both the technical infrastructure and the team are aligned with the organization’s goals. This position involves collaborating closely with the Election Technology Director to oversee the...",NaN
8,Language Access & Outreach Coordinator,https://www.governmentjobs.com/careers/kingcounty/jobs/4788084/language-access-and-outreach-coordinator-chinese?department[0]=KCE%20-%20King%20County%20Elections&sort=PositionTitle%7CAscending&pagetype=jobOpportunitiesJobs,"This is an amazing opportunity to be engaged in the election process! The Department of Elections is searching for energetic and resourceful professionals who like to “get stuff done”. The Language Access and Outreach Coordinator position in the Elections Department combines an exciting, fast 